<a href="https://colab.research.google.com/github/aniquetahir/Colaboratory/blob/master/MultiAspectNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/pmin91/DS_projects.git

Cloning into 'DS_projects'...
remote: Enumerating objects: 642, done.
remote: Total 642 (delta 0), reused 0 (delta 0), pack-reused 642
Receiving objects: 100% (642/642), 124.63 MiB | 25.62 MiB/s, done.
Resolving deltas: 100% (290/290), done.
Checking out files: 100% (298/298), done.


In [0]:
cd /content/DS_projects/04-Aspect_Based_Opinion_Mining/code

/content/DS_projects/04-Aspect_Based_Opinion_Mining/code


In [0]:
import xml.etree.ElementTree as ET
tree = ET.parse('../data/Restaurants_Train.xml')
root = tree.getroot()

In [0]:
import pandas as pd

In [0]:
# Use this dataframe for multilabel classification
# Must use scikitlearn's multilabel binarizer

labeled_reviews = []
for sentence in root.findall("sentence"):
    entry = {}
    aterms = []
    aspects = []
    if sentence.find("aspectTerms"):
        for aterm in sentence.find("aspectTerms").findall("aspectTerm"):
            aterms.append(aterm.get("term"))
    if sentence.find("aspectCategories"):
        for aspect in sentence.find("aspectCategories").findall("aspectCategory"):
            aspects.append(aspect.get("category"))
    entry["text"], entry["terms"], entry["aspects"]= sentence[0].text, aterms, aspects
    labeled_reviews.append(entry)
labeled_df = pd.DataFrame(labeled_reviews)
print("there are",len(labeled_reviews),"reviews in this training set")
#    print(sentence.find("aspectCategories").findall("aspectCategory").get("category"))

there are 3044 reviews in this training set


In [0]:
# Save annotated reviews
labeled_df.to_pickle("annotated_reviews_df.pkl")
labeled_df.head()

,text,terms,aspects
0,But the staff was so horrible to us.,[staff],[service]
1,"To be completely fair, the only redeeming fact...",[food],"[food, anecdotes/miscellaneous]"
2,"The food is uniformly exceptional, with a very...","[food, kitchen, menu]",[food]
3,Where Gabriela personaly greets you and recomm...,[],[service]
4,"For those that go once and don't enjoy it, all...",[],[anecdotes/miscellaneous]


# Training the model with Naive Bayes
1. replace pronouns with neural coref
2. train the model with naive bayes

In [0]:
!pip uninstall -y neuralcoref
!pip install --quiet neuralcoref --no-binary neuralcoref

In [0]:
!pip uninstall -y en-core-web-lg
!pip uninstall -y spacy
!pip install --quiet spacy==2.1.0

In [0]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 826.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255076 sha256=68a1dc3c928daea100a47422ba0a5b34f70f910d31049f291e8ac714f8d278c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-mso22wdc/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
  Found existing installation: en-core-web-lg 2.2.0
    Uninstalling en-core-web-lg-2.2.0:
      Successfully uninstalled en-core-web-lg-2.2.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import neuralcoref
import spacy
import en_core_web_lg


nlp = spacy.load('en_core_web_lg')
neuralcoref.add_to_pipe(nlp)

def replace_pronouns(text):
    doc = nlp(text)
    return doc._.coref_resolved

In [0]:
annotated_reviews_df = pd.read_pickle("../pickled_files/annotated_reviews_df.pkl")
annotated_reviews_df.head(3)

,aspects,terms,text
0,[service],[staff],But the staff was so horrible to us.
1,"[food, anecdotes/miscellaneous]",[food],"To be completely fair, the only redeeming fact..."
2,[food],"[food, kitchen, menu]","The food is uniformly exceptional, with a very..."


In [0]:
# Create a new column for text whose pronouns have been replaced
annotated_reviews_df["text_pro"] = annotated_reviews_df.text.map(lambda x: replace_pronouns(x))

In [0]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# Convert the multi-labels into arrays
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(annotated_reviews_df.aspects)
X = annotated_reviews_df.text_pro

# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0)

# save the the fitted binarizer labels
# This is important: it contains the how the multi-label was binarized, so you need to
# load this in the next folder in order to undo the transformation for the correct labels.
filename = 'mlb.pkl'
with open(filename, 'wb') as mlb_file:
    pickle.dump(mlb, mlb_file)

In [0]:
!pip install --quiet scikit-multilearn

     |████████████████████████████████| 92kB 4.0MB/s 


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from skmultilearn.problem_transform import LabelPowerset
import numpy as np

# LabelPowerset allows for multi-label classification
# Build a pipeline for multinomial naive bayes classification
text_clf = Pipeline([('vect', CountVectorizer(stop_words = "english",ngram_range=(1, 1))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', LabelPowerset(MultinomialNB(alpha=1e-1))),])
text_clf = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)

# Calculate accuracy
np.mean(predicted == y_test)

0.8651773981603154

In [0]:
# Test if SVM performs better
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', LabelPowerset(
                             SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, max_iter=6, random_state=42)))])
_ = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)

#Calculate accuracy
np.mean(predicted_svm == y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


0.8670170827858081

In [0]:
import pickle
# Train naive bayes on full dataset and save model
text_clf = Pipeline([('vect', CountVectorizer(stop_words = "english",ngram_range=(1, 1))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', LabelPowerset(MultinomialNB(alpha=1e-1))),])
text_clf = text_clf.fit(X, y)

# save the model to disk
filename = 'naive_model1.pkl'
with open(filename, 'wb') as model_file:
    pickle.dump(text_clf, model_file)

In [0]:
#mlb.inverse_transform(predicted)
pred_df = pd.DataFrame(
    {'text_pro': X_test,
     'pred_category': mlb.inverse_transform(predicted)
    })

In [0]:
pd.set_option('display.max_colwidth', -1)
pred_df.head()

,text_pro,pred_category
453,It's better than being on the roof of Sutton Place with 19 year old interns jabbing you in the ribs all night.,"(anecdotes/miscellaneous,)"
1611,"Don't expect to sit down inside though, there are only a few tables and they are always full.","(ambience,)"
2078,"Again, if you are in this neighborhood - by all means, come here.","(anecdotes/miscellaneous,)"
2715,Go there to relax and feel like your somewhere else.,"(anecdotes/miscellaneous,)"
2602,"As far as the service goes, the waitresses were not particularly friendly, but the waitresses got the job done.","(service,)"
